In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import requests,csv

import xlrd
import xlwt

from urllib import urlopen, quote

import json
import urllib2
import sys

from pyecharts import Geo
import pickle
%load_ext autoreload
%autoreload 2
# reload(sys)
# sys.setdefaultencoding('utf-8')

In [4]:
sumbox = pd.read_excel("cinema_movie.xls",sheetname=13)

In [5]:
Sumbox = sumbox[:100]
sumbox[:1]

,MoiveType,SumBoxOffice,CinemaID,CinemaName,CinemaAddress,CityCName
0,惊悚,11111010,1216,美嘉欢乐影城中关村店,海淀区中关村广场购物中心津乐汇三层（鼎好一期西侧）,海淀区


## 影院--票房

In [6]:
cinema_sumbox = {}

for _,row in Sumbox.iterrows():
    cinema = row['CinemaName']
    box = row['SumBoxOffice']
    cinema_sumbox[cinema] = box

#字典
cinema_sumbox

In [8]:
def getlnglat(address):
    url = 'http://api.map.baidu.com/geocoder/v2/'
    output = 'json'
    ak = 'hatbvFb8awXC88SSZqf0N9k92mCk44c0'
    add = quote(address) 
    uri = url + '?' + 'address=' + add  + '&output=' + output + '&ak=' + ak
    req = urlopen(uri)
    res = req.read().decode('utf8') 
    temp = json.loads(res)
    if not temp.has_key('result'):
        print address
        return [0,0]
    lat=temp['result']['location']['lat']
    lng=temp['result']['location']['lng']
    return [lng,lat]

getlnglat('和平区滨江道143号')

In [9]:
geo4cinema = {}
for _,row in Sumbox.iterrows():
    cinema = row['CinemaName']
    cinemaADD = row['CinemaAddress']
    geo4cinema[cinema] =getlnglat(cinemaADD.encode('utf8'))
    

### 热力图

In [332]:
import random
geo = Geo("不同类型全国电影票房Top100影院分布", "黑色幽默类", title_color="#fff", title_pos="center",
          width=1200, height=600, background_color='#404a59')
#attr, value = geo.cast(data)
geo.add("", geo4cinema.keys(),
       [cinema_sumbox[ row['CinemaName']] for _,row in Sumbox.iterrows()],
        type="heatmap", 
        is_visualmap=True,
        visual_range=[10000, 10000000],
        visual_text_color='#556377',
        geo_cities_coords=geo4cinema)
geo.render('heatmap_top100_sheet49.html')

In [10]:
geo = Geo("不同类型全国电影票房Top100影院分布", "惊悚类", title_color="#fff", title_pos="center",
          width=1200, height=600, background_color='#404a59')
# attr, value = geo.cast(data)
geo.add("", geo4cinema.keys(),
        [cinema_sumbox[ row['CinemaName']] for _,row in Sumbox.iterrows()],
        visual_range=[10000, 10000000], 
        visual_text_color="#fff", 
        symbol_size=15,
        is_visualmap=True, 
        geo_cities_coords=geo4cinema)
geo.render('view_top100_sheet13.html') #文件名啦！

In [105]:
for _,row in Sumbox.iterrows():
    print cinema_sumbox[ row['CinemaName']]

217957534
172823970
159482192
157898494
157830966
157575040
154572348
153383776
142840801
136969029
136372721
135543828
134219883
133036259
132770095
131903502
130669100
130168297
128326495
126984796
126779916
123405480
123338021
122887038
122205904
121346542
120556726
119406793
119090643
118637752
117994461
117932441
115312140
114881350
114728153
113338645
112263808
111744368
108564268
108331049
107730262
106842526
106791851
106437961
106430670
106394861
106176277
105640596
105104130
104753292
104618420
104516228
104162750
103918080
103910245
103500235
103482173
102913870
102275317
102130591
102119441
101767279
101641306
100078413
100042294
99909248
99559515
98933658
98440031
97333088
96793897
96142162
95908284
95529309
95220974
95092643
95051056
94860642
94141159
94093859
93746027
92923514
92812072
92665815
92586865
92557015
92284981
91847253
91809598
91741295
91737683
91383986
91370246
91170503
90901482
90872080
90856306
90852172
90479067
90303520


In [103]:
print velue

90303520


# 分析表格

In [3]:
actor[:2]

,ActorID,ActorCName,ActorEName,ActorNation
0,19,马晓晴,Xiaoqing Ma,中国
1,22,张磊,Lei Zhang,中国


In [4]:
movie[:6]

,MovieID,MovieCName,MovieSumBoxOffice,MovieType,MovieStandard
0,273,舞吧！昴,471000.0,歌舞/剧情,2D
1,718,焦裕禄,4090000.0,纪录片/传记,2D
2,852,寻龙夺宝,32116000.0,动作/冒险,2D
3,6348,失业生,6066000.0,剧情/青春/爱情,2D
4,7954,刺客聂隐娘,61405000.0,动作/武侠/古装,2D
5,8350,如梦,873000.0,爱情,2D


In [5]:
movie_actor[:2]

,MovieID,ActorID,ActorRank,ActorRole
0,273,2288,1,actor
1,273,2289,2,actor


In [6]:
Actor1 = pd.merge(pd.merge(movie_actor,actor),movie)
Actor1[:6]

,MovieID,ActorID,ActorRank,ActorRole,ActorCName,ActorEName,ActorNation,MovieCName,MovieSumBoxOffice,MovieType,MovieStandard
0,273,2288,1,actor,黑木明纱,Meisa Kuroki,NaN,舞吧！昴,471000.0,歌舞/剧情,2D
1,273,2289,2,actor,桃井薰,Kaori Momoi,NaN,舞吧！昴,471000.0,歌舞/剧情,2D
2,273,2290,3,actor,佐野光来,Miku Sano,NaN,舞吧！昴,471000.0,歌舞/剧情,2D
3,273,1862492,1,director,李志毅,Li Chi Ngai,中国香港,舞吧！昴,471000.0,歌舞/剧情,2D
4,273,2014387,4,actor,黑木梅纱,HeiMuMeiMiao,日本,舞吧！昴,471000.0,歌舞/剧情,2D
5,273,2014388,5,actor,高雅罗,GaoYaLuo,中国,舞吧！昴,471000.0,歌舞/剧情,2D


## 用电影来链接演员的信息，就会出现演员和

In [10]:
Actor2 = pd.merge(pd.merge(movie,movie_actor),actor)
print Actor2.size
Actor2[:6]

135894


,MovieID,MovieCName,MovieSumBoxOffice,MovieType,MovieStandard,ActorID,ActorRank,ActorRole,ActorCName,ActorEName,ActorNation
0,273,舞吧！昴,471000.0,歌舞/剧情,2D,2288,1,actor,黑木明纱,Meisa Kuroki,NaN
1,663007,青禾男高,30409000.0,爱情/动作/喜剧,2D,2288,8,actor,黑木明纱,Meisa Kuroki,NaN
2,273,舞吧！昴,471000.0,歌舞/剧情,2D,2289,2,actor,桃井薰,Kaori Momoi,NaN
3,657795,攻壳机动队,202559000.0,科幻/动作,3D/IMAX,2289,7,actor,桃井薰,Kaori Momoi,NaN
4,273,舞吧！昴,471000.0,歌舞/剧情,2D,2290,3,actor,佐野光来,Miku Sano,NaN
5,273,舞吧！昴,471000.0,歌舞/剧情,2D,1862492,1,director,李志毅,Li Chi Ngai,中国香港


In [11]:
Actor = pd.merge(pd.merge(actor,movie_actor),movie)
print Actor.size
Actor[:20]

135894


,ActorID,ActorCName,ActorEName,ActorNation,MovieID,ActorRank,ActorRole,MovieCName,MovieSumBoxOffice,MovieType,MovieStandard
0,19,马晓晴,Xiaoqing Ma,中国,611390,4,actor,团圆,362000.0,爱情/亲情,2D
1,5563,王全安,Quanan Wang,中国,611390,1,director,团圆,362000.0,爱情/亲情,2D
2,6312,卢燕,Lisa Lu,中国,611390,1,actor,团圆,362000.0,爱情/亲情,2D
3,10675,莫小棋,Monica Mok,澳大利亚,611390,3,actor,团圆,362000.0,爱情/亲情,2D
4,1867934,徐才根,Caigen Xu,中国,611390,5,actor,团圆,362000.0,爱情/亲情,2D
5,2187516,凌峰,Feng Ling,NaN,611390,2,actor,团圆,362000.0,爱情/亲情,2D
6,22,张磊,Lei Zhang,中国,612723,3,actor,推拿,13148000.0,爱情,2D
7,4622,梅婷,Ting Mei,中国,612723,4,actor,推拿,13148000.0,爱情,2D
8,5461,黄璐,Lu Huang,中国,612723,6,actor,推拿,13148000.0,爱情,2D
9,6138,郭晓东,Xiaodong Guo,中国,612723,1,actor,推拿,13148000.0,爱情,2D
